In [1]:
# library
import torch
import numpy as np
import pandas as pd
from torchmetrics import R2Score

r2score = R2Score()

torch.manual_seed(1)
np.random.seed(1)

In [2]:
# Model
class MonotonicLinear(torch.nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.input_size  = input_size
    self.output_size = output_size
    self.weights = torch.nn.Parameter(torch.rand((output_size, input_size), dtype=torch.float32))

  def forward(self, x):
    z = torch.mm(x, torch.exp(self.weights.t()))
    return z

class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.input1  = MonotonicLinear(2, 32)
    self.hidden1 = MonotonicLinear(32, 1)
    self.input2  = torch.nn.Linear(2, 32)
    self.hidden2 = torch.nn.Linear(32, 1)

  def forward(self, x_m, x_u):
    z1 = torch.tanh(self.input1(x_m))
    z1 = self.hidden1(z1)

    z2 = torch.tanh(self.input2(x_u))
    z2 = self.hidden2(z2)

    z = torch.add(z1, z2)
    return z

# Create network
device = torch.device("cpu")
net = Net().to(device)

In [3]:
# Model evaluation
def eval(model, testset):
    with torch.no_grad():
        pred_Y = model(testset.x_m_data, testset.x_u_data)
    
    r2 = r2score(pred_Y, testset.y_data)
    return r2.item()

In [4]:
# Data class
class Data(torch.utils.data.Dataset):
  def __init__(self, src_file, start=None, end=None):
    df = pd.read_csv(src_file)
    Tca_k1   = np.array(df['ahu_supply_temp']).reshape(-1,1)[start+1: end+1]
    Tsa_k    = np.array(df['supply_discharge_temp']).reshape(-1,1)[start: end]
    Tsa_k1   = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+1: end+1]
    valve_k2 = np.array(df['htg_valve_position']).reshape(-1,1)[start+2: end+2]

    tmp_x_m = np.concatenate((Tca_k1, valve_k2), axis=1)
    tmp_x_u = np.concatenate((Tsa_k, Tsa_k1), axis=1)
    tmp_y   = np.array(df['supply_discharge_temp']).reshape(-1,1)[start+2: end+2]

    self.x_m_data = torch.tensor(tmp_x_m, dtype=torch.float32)
    self.x_u_data = torch.tensor(tmp_x_u, dtype=torch.float32)
    self.y_data = torch.tensor(tmp_y, dtype=torch.float32)

  def __len__(self):
    return len(self.y_data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    inp_m  = self.x_m_data[idx]
    inp_u  = self.x_u_data[idx]
    outp   = self.y_data[idx]
    sample = {'inp_m':inp_m,'inp_u':inp_u, 'outp':outp}
    return sample

In [5]:
# Early stopping function
def early_stop(list, min_epochs, patience):
    if(len(list) > min_epochs):
        if(np.max(list[-patience:]) < 1.00001*np.max(list[0: -patience])):
            return 1
    return 0

In [6]:
# train function
def train(net, train_ds, test_ds, lr=0.001, min_epochs=200, max_epochs=100000, patience=100, smooth=0):
    loss_func  = torch.nn.MSELoss()
    optimizer  = torch.optim.Adam(net.parameters(), lr=lr)

    R2_test    = np.array([])
    train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=train_ds.y_data.shape[0], shuffle=True)
    for _ in range(0, max_epochs+1):
        net.train()
        for (_, batch) in enumerate(train_ldr):
            X_m = batch['inp_m']
            X_u = batch['inp_u']
            Y   = batch['outp']

            optimizer.zero_grad()
            output = net(X_m, X_u)
            loss_val = loss_func(output, Y) + smooth*loss_func(output, X_u[:, 1].reshape(-1,1))
            loss_val.backward()
            optimizer.step()

        net.eval()
        R2_test = np.append(R2_test, eval(net, test_ds))
        
        if(early_stop(list = R2_test, min_epochs = min_epochs, patience = patience) == 1):
            break
    
    return R2_test

In [7]:
# main
for _n_train in [32, 64, 128]:
    for _smooth in [0, 0.001, 0.01]:
        for _lr in [0.0001, 0.001, 0.01, 0.1, 0.2]:

            # Create network
            device = torch.device("cpu")
            net = Net().to(device)

            # Create Dataset and DataLoader objects
            src_file = 'C:/Users/tln229/Downloads/Python/1. Building/data/HVAC_B90_102_exp_10m_20210424.csv'
            n_train  = _n_train
            train_ds = Data(src_file, start=0,       end=n_train)
            test_ds  = Data(src_file, start=n_train, end=1600)

            # train
            R2_test = train(net, train_ds, test_ds, lr=_lr, min_epochs=1000, max_epochs=50000, patience=500, smooth=_smooth)

            # results
            print('n train = %3d \t smooth = %6.4f \t lr = %6.4f \t best_epoch = %5d \t best_R2 = %7.5f'
                % (_n_train, _smooth, _lr, np.argmax(R2_test), np.max(R2_test)))

n train =  32 	 smooth = 0.0000 	 lr = 0.0001 	 best_epoch =  4615 	 best_R2 = -0.42982
n train =  32 	 smooth = 0.0000 	 lr = 0.0010 	 best_epoch =  8764 	 best_R2 = 0.95200
n train =  32 	 smooth = 0.0000 	 lr = 0.0100 	 best_epoch =    34 	 best_R2 = -0.18451
n train =  32 	 smooth = 0.0000 	 lr = 0.1000 	 best_epoch =     2 	 best_R2 = -0.00000
n train =  32 	 smooth = 0.0000 	 lr = 0.2000 	 best_epoch =    11 	 best_R2 = -0.00814
n train =  32 	 smooth = 0.0010 	 lr = 0.0001 	 best_epoch =  3816 	 best_R2 = -0.41829
n train =  32 	 smooth = 0.0010 	 lr = 0.0010 	 best_epoch = 15630 	 best_R2 = 0.94116
n train =  32 	 smooth = 0.0010 	 lr = 0.0100 	 best_epoch =  1563 	 best_R2 = 0.83123
n train =  32 	 smooth = 0.0010 	 lr = 0.1000 	 best_epoch =     3 	 best_R2 = -0.00025
n train =  32 	 smooth = 0.0010 	 lr = 0.2000 	 best_epoch =     1 	 best_R2 = -0.00731
n train =  32 	 smooth = 0.0100 	 lr = 0.0001 	 best_epoch =  7814 	 best_R2 = -0.39441
n train =  32 	 smooth = 0.0100 	 l